<a href="https://colab.research.google.com/github/beinghorizontal/deepface/blob/main/DFL_Colab_auto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to DFL-Colab!

This is an adapted version of the DFL for Google Colab.


# Overview
*   Extractor works in full functionality.
*   Training can work without preview.
*   Merger works in full functionality.
*   You can import/export workspace with your Google Drive.
*   Import/export and another manipulations with workspace you can do in "Manage workspace" block
*   Google Colab machine active for 12 hours. DFL-Colab makes a backup of your workspace in training mode.
*   Google does not like long-term heavy calculations. Therefore, for training more than two sessions in a row, use two Google accounts. It is recommended to split your training over 2 accounts, but you can use one Google Drive account to store your workspace.



## Prevent random disconnects

This cell runs JS code to automatic reconnect to runtime.

In [ ]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

## Check GPU

*   Google Colab can provide you with one of Tesla graphics cards: K80, T4, P4 or P100
*   Here you can check the model of GPU before using DeepFaceLab



In [ ]:
!nvidia-smi

## Install or update DeepFaceLab

* Install or update DeepFAceLab directly from Github
* Requirements install is automatically
* Automatically sets timer to prevent random disconnects
* "Download FFHQ" option means to download high quality FFHQ dataset instead of CelebA. FFHQ takes up more memory, so it will take longer to download than CelebA. It is recommended to enable this option if you are doing pretrain.

In [ ]:
!apt-get update
!apt-get install -y libsm6 libxext6 libxrender-dev
!pip install jedi
!pip install --upgrade pip
!pip install setuptools==64.0.2



In [ ]:
#!pip uninstall -y opencv-python
#!pip install opencv-python==4.5.1.48


In [ ]:
#@title Install or update DeepFaceLab from Github

Mode = "install" #@param ["install", "update"]
Download_FFHQ = False #@param {type:"boolean"}


pretrain_link = "https://github.com/beinghorizontal/deep_fc/releases/download/"
pretrain_link = pretrain_link+"pretrain_GenericFFHQ/pretrain_FFHQ.zip" if Download_FFHQ else pretrain_link+"pretrain-CelebA/pretrain_CelebA.zip"

from pathlib import Path
if (Mode == "install"):
  #!git clone https://github.com/iperov/DeepFaceLab.git
  !git clone https://github.com/beinghorizontal/deep_fc.git
  #%cd "/content/deep_fc"
  #!git checkout 9ad9728b4021d1dff62905cce03e2157d0c0868d
  #%cd "/content"

  # fix linux warning
  # /usr/lib/python3.6/multiprocessing/semaphore_tracker.py:143: UserWarning: semaphore_tracker: There appear to be 1 leaked semaphores to clean up at shutdown
  #fin = open("/usr/lib/python3.6/multiprocessing/semaphore_tracker.py", "rt")
  #data = fin.read()
  #data = data.replace('if cache:', 'if False:')
  #fin.close()

  #fin = open("/usr/lib/python3.6/multiprocessing/semaphore_tracker.py", "wt")
  #fin.write(data)
  #fin.close()
else:
  %cd /content/deep_fc
  !git pull

!pip uninstall -y tensorflow
!pip install -r /content/deep_fc/requirements-colab.txt
#!pip install --upgrade scikit-image
!apt-get install cuda-11.2

#if not Path("/content/pretrain").exists():
#  print("Downloading Pretrain faceset ... ")
#  #!wget -q --no-check-certificate -r $pretrain_link -O /content/pretrain_faceset.zip
#  !mkdir /content/pretrain
#  !unzip -q /content/pretrain_faceset.zip -d /content/pretrain/
#  !rm /content/pretrain_faceset.zip

#if not Path("/content/pretrain_Q96").exists():
#  print("Downloading Q96 pretrained model ...")
#  !wget -q --no-check-certificate -r 'https://github.com/chervonij/DFL-Colab/releases/download/Q96_model_pretrained/Q96_model_pretrained.zip' -O /content/pretrain_Q96.zip
#  !mkdir /content/pretrain_Q96
#  !unzip -q /content/pretrain_Q96.zip -d /content/pretrain_Q96/
#  !rm /content/pretrain_Q96.zip

if not Path("/content/workspace").exists():
  !mkdir /content/workspace; mkdir /content/workspace/data_src; mkdir /content/workspace/data_src/aligned; mkdir /content/workspace/data_dst; mkdir /content/workspace/data_dst/aligned; mkdir /content/workspace/model  

import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("\nDone!")

## Manage workspace



*   You can import/export workspace or individual data, like model files with Google Drive
*   Also, you can use HFS (HTTP Fileserver) for directly import/export you workspace from your computer
*   You can clear all workspace or delete part of it



In [ ]:
#@title Import from Drive

Mode = "workspace" #@param ["workspace", "data_src", "data_dst", "data_src aligned", "data_dst aligned", "models"]
Archive_name = "workspace.zip" #@param {type:"string"}

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive')

def zip_and_copy(path, mode):
  unzip_cmd=" -q "+Archive_name
  
  %cd $path
  copy_cmd = "/content/drive/My\ Drive/"+Archive_name+" "+path
  !cp $copy_cmd
  !unzip $unzip_cmd    
  !rm $Archive_name

if Mode == "workspace":
  zip_and_copy("/content", "workspace")
elif Mode == "data_src":
  zip_and_copy("/content/workspace", "data_src")
elif Mode == "data_dst":
  zip_and_copy("/content/workspace", "data_dst")
elif Mode == "data_src aligned":
  zip_and_copy("/content/workspace/data_src", "aligned")
elif Mode == "data_dst aligned":
  zip_and_copy("/content/workspace/data_dst", "aligned")
elif Mode == "models":
  zip_and_copy("/content/workspace", "model")
  
print("Done!")



## Train model

* Choose your model type, but SAEHD is recommend for everyone
* Set model options on output field
* You can see preview manually, if go to model folder in filemanager and double click on preview.jpg file
* Your workspace will be archived and upload to mounted Drive after 11 hours from start session
* If you select "Backup_every_hour" option, your workspace will be backed up every hour.
* Also, you can export your workspace manually in "Manage workspace" block
* "Silent_Start" option provides to automatically start with best GPU and last used model. 

In [ ]:
#@title Training
Model = "SAEHD" #@param ["SAEHD", "AMP", "Quick96", "XSeg"]
Backup_every_hour = True #@param {type:"boolean"}
Silent_Start = True #@param {type:"boolean"}

%cd "/content"

#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive')

import psutil, os, time

p = psutil.Process(os.getpid())
uptime = time.time() - p.create_time()

if (Backup_every_hour):
  if not os.path.exists('workspace.zip'):
    print("Creating workspace archive ...")
    !zip -r -q workspace.zip workspace
    print("Archive created!")
  else:
    print("Archive exist!")

if (Backup_every_hour):
  print("Time to end session: "+str(round((43200-uptime)/1200))+" hours")
  backup_time = str(1200)
  backup_cmd = " --execute-program -"+backup_time+" \"import os; os.system('zip -r -q workspace.zip workspace/model'); os.system('cp /content/workspace.zip /content/drive/My\ Drive/'); print('Backed up!') \"" 
elif (round(39600-uptime) > 0):
  print("Time to backup: "+str(round((39600-uptime)/1200))+" hours")
  backup_time = str(round(39600-uptime))
  backup_cmd = " --execute-program "+backup_time+" \"import os; os.system('zip -r -q workspace.zip workspace'); os.system('cp /content/workspace.zip /content/drive/My\ Drive/'); print('Backed up!') \"" 
else:
  print("Session expires in less than an hour.")
  backup_cmd = ""
    
cmd = "deep_fc/main.py train --training-data-src-dir workspace/data_src/aligned --training-data-dst-dir workspace/data_dst/aligned --pretraining-data-dir pretrain --model-dir workspace/model --model "+Model

if Model == "Quick96":
  cmd+= " --pretrained-model-dir pretrain_Q96"

if Silent_Start:
  cmd+= " --silent-start"

if (backup_cmd != ""):
  train_cmd = (cmd+backup_cmd)
else:
  train_cmd = (cmd)

!python $train_cmd

# Unzip drive content

In [ ]:
#!unzip /content/drive/MyDrive/data_dst_dp.zip -d /content/workspace/data_dst

# Remove folders

In [ ]:
#!rm -rf "/content/workspace/model"